In [8]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import spacy
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_20newsgroups

model = SentenceTransformer('all-mpnet-base-v2')

In [9]:
# topic_model = BERTopic( embedding_model=model)
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
df_gpt_responses = pd.read_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_processed_data_main.json")
df_gpt_responses.head()

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens,Neutral_Encodings,Positive_Encodings,Negative_Encodings,Neutral_Terminated,Neutral_Terminated_Encodings,Positive_Leading,Positive_Leading_Encodings,Negative_Leading,Negative_Leading_Encodings
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91,"[0.052540335800000004, -0.0741394237, -0.02414...","[0.0398985222, -0.039076753000000006, -0.03879...","[0.0750338361, 0.055929131800000004, -0.034461...",I am a Technical Support Representative.,"[0.0467863977, -0.0582639724, -0.0304021016000...",I am knowledgeable and skilled in providing te...,"[0.0400350429, -0.0512237102, -0.0361330472000...",I may encounter difficult and frustrated custo...,"[0.06889098140000001, 0.053461976300000005, -0..."
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100,"[-0.0183508322, 0.025549562600000002, -0.02162...","[-0.0315244012, 0.0867658779, -0.0072295303, -...","[-0.027420181800000002, 0.0644791871, -0.00949...",I am an A&H Underwriter .,"[-0.022203857100000002, 0.0287630353, -0.02035...",I have a great job as an A&H Underwriter,"[-0.0302970372, 0.0902502164, -0.0129407654, -...",I am not happy with my job as an A&H Underwriter,"[-0.026900822300000002, 0.0706868768, -0.01072..."
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109,"[-0.017012547700000002, 0.0080151027, -0.02787...","[-0.0027675340000000003, 0.068974562, -0.00189...","[-0.0194694344, 0.0587330759, -0.0063610324, -...",I am an A&H Underwriting Manager.,"[-0.0162954777, 0.0157897249, -0.0263584778000...",I have a high-level position as an A&H Underwr...,"[-0.0147530027, 0.0676713511, 0.0007165152, -0...","I am not just an entry-level employee, but I h...","[-0.0198444836, 0.054371766700000004, -0.00303..."
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118,"[-0.040869303, -0.0479970723, -0.0461201258, -...","[-0.027997929600000002, 0.0399987362, -0.03595...","[-0.0378840007, -0.030123831700000003, -0.0423...",I am an A&P Technician .,"[-0.0417694934, -0.0297952704, -0.0478198938, ...",Being an A&P Technician is a rewarding career ...,"[-0.028020246000000002, 0.0396952815, -0.03876...",The job of an A&P Technician can be physically...,"[-0.0376795791, -0.030884342300000003, -0.0492..."
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100,"[0.0211608782, -0.0087794699, -0.0511314794, -...","[0.014801985600000001, 0.0252737999, -0.049951...","[0.0058508902, -0.0159540195, -0.0475744903, -...",I am an ABA Therapist.,"[0.015887849000000002, 0.0088945329, -0.048975...",I provide effective therapy to children with a...,"[0.0150047606, 0.0090112872, -0.05012279, 0.01...",I do not enjoy working as an ABA Therapist and...,"[0.0030206721000000003, -0.0278182309, -0.0552..."


In [18]:
def filter_text(df: pd.DataFrame, score_num):
    return df[df["Score"] == score_num]["Text"].tolist()

def get_all_text(df_list, score_num):
    text = []
    for df in df_list:
        text.extend(filter_text(df, score_num))
    return text


positive_text = df_gpt_responses["Positive"].tolist()
negative_text = df_gpt_responses["Negative"].tolist()

print(len(positive_text))
print(len(negative_text))

13447
13447


In [25]:
dim_model = PCA(n_components=5)

topic_model_pos = BERTopic(positive_text, embedding_model=model, umap_model=dim_model)
topic_model_neg = BERTopic(negative_text, embedding_model=model, umap_model=dim_model)

topics_pos, probs_pos = topic_model_pos.fit_transform(positive_text)
topics_neg, probs_neg = topic_model_neg.fit_transform(negative_text)

In [26]:
print(len(topics_pos))
print(len(topics_neg))

13447
13447


In [32]:
df_gpt_responses["Positive_Topic"] = topics_pos
df_gpt_responses["Negative_Topic"] = topics_neg

df_gpt_responses.to_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_processed_data_main.json")

: 

In [47]:
topic_model.save("../Data_Storage/BERTopic/gpt_pca_responses_positive")